In [49]:
#GET THE SHAPEFILE OF BLOCK GROUPS IN NEW YORK STATE FROM .SHP's DBF FILE TO A CSV
import csv
from dbfpy import dbf
import os
import sys
 
filename = '/Users/danielmsheehan/Dropbox/GIS/Data/Census/census_2010/block_group/tl_2015_36_bg/tl_2015_36_bg.dbf'

if filename.endswith('.dbf'):
    print "Converting %s to csv" % filename
    csv_fn = filename[:-4]+ ".csv"
    with open(csv_fn,'wb') as csvfile:
        in_db = dbf.Dbf(filename)
        out_csv = csv.writer(csvfile)
        names = []
        for field in in_db.header.fields:
            names.append(field.name)
        out_csv.writerow(names)
        for rec in in_db:
            out_csv.writerow(rec.fieldData)
        in_db.close()
        print "Done..."
else:
  print "Filename does not end with .dbf"
#from https://gist.github.com/bertspaan/8220892

Converting /Users/danielmsheehan/Dropbox/GIS/Data/Census/census_2010/block_group/tl_2015_36_bg/tl_2015_36_bg.dbf to csv
Done...


In [51]:
import pandas as pd 

theBlockGroupCSV = filename.replace('.dbf','.csv')

df = pd.read_csv(theBlockGroupCSV)

dfBlockGroup = df[(df.COUNTYFP == 59)] #County 059 is Nassau County

dfBlockGroup.head(10)

,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON
1232,36,59,303700,3,360593037003,Block Group 3,G5030,S,641365,0,40.746295,-73.627806
1233,36,59,406501,4,360594065014,Block Group 4,G5030,S,710982,0,40.721855,-73.627570
1234,36,59,407600,3,360594076003,Block Group 3,G5030,S,188905,0,40.747186,-73.559084
1235,36,59,408500,2,360594085002,Block Group 2,G5030,S,359117,0,40.702905,-73.533934
1236,36,59,409500,2,360594095002,Block Group 2,G5030,S,428980,0,40.713864,-73.488764
1792,36,59,404400,4,360594044004,Block Group 4,G5030,S,173425,0,40.721153,-73.713072
1793,36,59,404500,2,360594045002,Block Group 2,G5030,S,329924,0,40.725300,-73.696306
1794,36,59,404700,1,360594047001,Block Group 1,G5030,S,424798,0,40.730916,-73.684759
1795,36,59,404700,3,360594047003,Block Group 3,G5030,S,228482,0,40.717291,-73.685356
1796,36,59,404901,3,360594049013,Block Group 3,G5030,S,175553,0,40.703370,-73.722574


In [52]:
import json
import urllib2
from pprint import pprint

listOfGeoids = dfBlockGroup["GEOID"].tolist()

variablesWanted = ['GEOID','TRANSIT_INDEX']

df = pd.DataFrame(columns=variablesWanted)

variablesURL = '%2C'.join(variablesWanted)

urlPart1 = 'http://egis.hud.gov/ArcGIS/rest/services/oshc/Fhea/MapServer/5/query?text=&geometry=&geometryType=esriGeometryPoint&inSR=&spatialRel=esriSpatialRelIntersects&relationParam=&objectIds=&where=GEOID+%3D+%27'
urlPart3 = '%27&time=&returnCountOnly=false&returnIdsOnly=false&returnGeometry=false&maxAllowableOffset=&outSR=&outFields='
urlPart5 = '&f=pjson'

for i in listOfGeoids: #maybe create larger lists of BlockGroup GEOIDS, hard b/c 1000 limit, and there's more than that of Block Groups
    #print i
    i = str(i)
    urlPart2 = i
    urlPart4 = variablesURL
    
    theURL = urlPart1 + urlPart2 + urlPart3+ urlPart4 + urlPart5
    #print theURL
    
    data = json.load(urllib2.urlopen(theURL))
    
    try:
        jsonGeoid = data[u'features'][0][u'attributes'][u'GEOID']
        jsonTransit_index = data[u'features'][0][u'attributes'][u'transit_index']
    
    except:
        jsonGeoid = i
        jsonTransit_index = -999
        
    print 'BlockGroup: ' + str(jsonGeoid) + ' - the transit index is: ' + str(jsonTransit_index)
    
    dfAppend = pd.DataFrame([[jsonGeoid, jsonTransit_index]], columns=columns) 
    df = df.append(dfAppend)
       
print df.head(10)

http://egis.hud.gov/ArcGIS/rest/services/oshc/Fhea/MapServer/5/query?text=&geometry=&geometryType=esriGeometryPoint&inSR=&spatialRel=esriSpatialRelIntersects&relationParam=&objectIds=&where=GEOID+%3D+%27360593037003%27&time=&returnCountOnly=false&returnIdsOnly=false&returnGeometry=false&maxAllowableOffset=&outSR=&outFields=GEOID%2CTRANSIT_INDEX&f=pjson
BlockGroup: 360593037003 - the transit index is: 80
http://egis.hud.gov/ArcGIS/rest/services/oshc/Fhea/MapServer/5/query?text=&geometry=&geometryType=esriGeometryPoint&inSR=&spatialRel=esriSpatialRelIntersects&relationParam=&objectIds=&where=GEOID+%3D+%27360594065014%27&time=&returnCountOnly=false&returnIdsOnly=false&returnGeometry=false&maxAllowableOffset=&outSR=&outFields=GEOID%2CTRANSIT_INDEX&f=pjson
BlockGroup: 360594065014 - the transit index is: 90
http://egis.hud.gov/ArcGIS/rest/services/oshc/Fhea/MapServer/5/query?text=&geometry=&geometryType=esriGeometryPoint&inSR=&spatialRel=esriSpatialRelIntersects&relationParam=&objectIds=&wh

error: [Errno 60] Operation timed out

In [53]:
df.to_csv('data/geoid_hud_transit_index.csv',index=False)

print df.head(1200)

           GEOID  TRANSIT_INDEX
0   360593037003             80
0   360594065014             90
0   360594076003             80
0   360594085002             70
0   360594095002              1
0   360594044004             70
0   360594045002             80
0   360594047001             80
0   360594047003             70
0   360594049013             80
0   360594050003            100
0   360594051001             80
0   360594052001             80
0   360594052003             80
0   360594053011            100
0   360594053025            100
0   360594054002             90
0   360594082003             70
0   360594086001             90
0   360594105002             90
0   360594118003            100
0   360595213024              1
0   360595206005              1
0   360594054006             80
0   360594055001              1
0   360594057003             80
0   360594054003             80
0   360594058003            100
0   360594059003             90
0   360594060014              1
..      

Example of a URL getting data from: WHERE GEOID = 360593028004

###[Transit Index Info] (http://egis.hud.gov/ArcGIS/rest/services/oshc/Fhea/MapServer/5)

http://egis.hud.gov/ArcGIS/rest/services/oshc/Fhea/MapServer/5/query?text=&geometry=&geometryType=esriGeometryPoint&inSR=&spatialRel=esriSpatialRelIntersects&relationParam=&objectIds=&where=GEOID+%3D+%27360593028004%27&time=&returnCountOnly=false&returnIdsOnly=false&returnGeometry=false&maxAllowableOffset=&outSR=&outFields=GEOID%2CTRANSIT_INDEX&f=pjson